# 数组（包括矩阵）

- (多维)数组: 由下标(index)和值(value)组成的序列的集合
- C++中有静态数组和动态数组；
- 从定义上来看，线性表和数组都是数据元素的有序集
- 数组有维度（比如三维数组）的概念而线性表没有
- 数组一般不进行数据插入和删除操作


## 矩阵压缩存储：
### 对称矩阵：
- 对称矩阵的压缩存储：只存储上三角或下三角部分
- 例如，存储上三角部分时，元素的存储位置可以通过
  - `i`行和`j`列的元素存储在位置 `i * (i + 1) / 2 + j` 中
### 稀疏矩阵：
- 稀疏矩阵的压缩存储：只存储非零元素
- 例如，使用三元组表示法存储非零元素的行、列和数值
#### 运算：
稀疏矩阵三元组表存储中元素的位置和下标没有关系，因此无法依靠下标进行矩阵运算
- 需新算法进行矩阵转置、矩阵求逆、矩阵加减、矩阵乘除等
- 慢速转置：逐趟扫描三元组序列，第k趟提取col值为k的三元组，放入目标压缩矩阵B
- 快速转置：先统计每列非零元素个数，建立col的分布表，然后按分布表将三元组放入目标压缩矩阵B
  - 辅助向量rowStart[ ]固定在稀疏矩阵的三元组表中，用来指示“行”的信息，得到另一种顺序存储结构：行逻辑链接的三元组顺序表
- 矩阵乘法：逐行扫描A的三元组表，逐列扫描B的三元组表，计算结果存入C的三元组表
  - 遍历A中任意非零元素, 其行列分别为i,k；
  - 在B中遍历搜索行号为 k 的任意元素相乘，结果累加入C[i][j] 
  - 遍历搜索行号为 k 的元素可由rowStart数组直接给出

### 十字链表
- 十字链表：用于存储稀疏矩阵的双向链表
- 每个非零元素用一个结点表示，结点包含行号、列号、值以及指向同一行和同一列的前驱和后继结点的指针
- 十字链表的优点是可以快速访问行和列的非零元素

In [2]:
from __future__ import annotations
from dataclasses import dataclass
from abc import ABC, abstractmethod
from typing import List, Iterable, Tuple


# ========= 基础与继承结构 =========
class SparseMatrixBase(ABC):
    """稀疏矩阵抽象基类，定义通用接口。"""
    def __init__(self, rows: int, cols: int, nnz: int = 0):
        if rows < 0 or cols < 0:
            raise ValueError("rows/cols must be non-negative")
        self._rows = rows
        self._cols = cols
        self._nnz = nnz

    @property
    def shape(self) -> Tuple[int, int]:
        return self._rows, self._cols

    @property
    def nnz(self) -> int:
        return self._nnz

    @abstractmethod
    def transpose_slow(self) -> "SparseMatrixBase":
        """O(rows * nnz) 级别的朴素转置（逐列扫描+遍历三元组）。"""
        ...

    @abstractmethod
    def transpose_fast(self) -> "SparseMatrixBase":
        """O(cols + nnz) 级别的快速转置（计数列元素 + 前缀和定位）。"""
        ...


@dataclass(frozen=True)
class Triple:
    """三元组：行、列、值（默认使用0-based索引）。"""
    r: int
    c: int
    v: float


class TripletMatrix(SparseMatrixBase):
    """
    三元组顺序表稀疏矩阵。
    - 使用 0-based 索引
    - 不强制按行或按列排序，但推荐按行主序存储
    """
    def __init__(self, rows: int, cols: int, triples: Iterable[Triple] = ()):
        triples_list = list(triples)
        super().__init__(rows, cols, nnz=len(triples_list))
        # 可做基本越界与零值过滤
        for t in triples_list:
            if not (0 <= t.r < rows and 0 <= t.c < cols):
                raise IndexError(f"triple out of bounds: {t}")
            # 保留零值与否按需；这里直接允许，但通常应过滤零值
        self.data: List[Triple] = triples_list

    # ========== 构造与辅助 ==========
    @classmethod
    def from_dense(cls, A: List[List[float]]) -> "TripletMatrix":
        rows = len(A)
        cols = 0 if rows == 0 else len(A[0])
        triples: List[Triple] = []
        for i in range(rows):
            if len(A[i]) != cols:
                raise ValueError("dense rows must have equal length")
            for j in range(cols):
                if A[i][j] != 0:
                    triples.append(Triple(i, j, A[i][j]))
        return cls(rows, cols, triples)

    def to_dense(self) -> List[List[float]]:
        r, c = self.shape
        dense = [[0 for _ in range(c)] for _ in range(r)]
        for t in self.data:
            dense[t.r][t.c] = t.v
        return dense

    # ========== 慢速转置（朴素法） ==========
    def transpose_slow(self) -> "TripletMatrix":
        """
        思路：逐列 j = 0..cols-1 扫描原三元组，将 A 中列为 j 的项 (i,j,v) 变为 (j,i,v) 追加到结果。
        复杂度：O(cols * nnz)（因为每列都要扫描一遍数据）。
        """
        rows, cols = self.shape
        out: List[Triple] = []
        # 逐列扫描
        for j in range(cols):
            for t in self.data:
                if t.c == j:
                    out.append(Triple(t.c, t.r, t.v))
        return TripletMatrix(cols, rows, out)

    # ========== 快速转置（计数+前缀和） ==========
    def transpose_fast(self) -> "TripletMatrix":
        """
        思路（经典快速转置）：
        1) 统计 A 每一列的非零数目 col_count[j]
        2) 计算每列在转置后顺序表中的起始位置 col_pos[j] = prefix_sum(col_count)[j]
        3) 按原顺序遍历 A 的三元组 (i,j,v)，将其放到 B 的 out[col_pos[j]] = (j,i,v)，并 col_pos[j] += 1
        复杂度：O(cols + nnz)
        """
        rows, cols = self.shape
        t = self.nnz
        out: List[Triple] = [Triple(0, 0, 0)] * t  # 先占位

        # 1) 统计每列元素数
        col_count = [0] * cols
        for tri in self.data:
            col_count[tri.c] += 1

        # 2) 计算每列起始位置（稳定放置）
        col_pos = [0] * cols
        running = 0
        for j in range(cols):
            col_pos[j] = running
            running += col_count[j]

        # 3) 放置元素
        out_mut: List[Triple] = [None] * t  # type: ignore
        for tri in self.data:
            j = tri.c
            idx = col_pos[j]
            out_mut[idx] = Triple(j, tri.r, tri.v)
            col_pos[j] += 1

        return TripletMatrix(cols, rows, out_mut)  # type: ignore
    
    # ========== 稀疏矩阵乘法 ==========
    def matmul(self, other: "TripletMatrix") -> "TripletMatrix":
        """
        稀疏矩阵乘法: C = A * B
        A: m×k, B: k×n, 返回 C: m×n
        """
        A = self
        B = other
        m, k1 = A.shape
        k2, n = B.shape
        if k1 != k2:
            raise ValueError(f"shape mismatch: {A.shape} * {B.shape}")

        result_dict = {}

        # 遍历 A 的每个三元组
        for ta in A.data:
            i, p, va = ta.r, ta.c, ta.v
            # 遍历 B 的每个三元组
            for tb in B.data:
                if tb.r == p:  # 只有 A 的列 == B 的行 才能相乘
                    j, vb = tb.c, tb.v
                    result_dict[(i, j)] = result_dict.get((i, j), 0) + va * vb

        # 过滤掉值为 0 的项
        triples = [Triple(i, j, v) for (i, j), v in result_dict.items() if v != 0]
        return TripletMatrix(m, n, triples)


# example usage
if __name__ == "__main__":
    # 稠密构造：3x4
    dense = [
        [0, 5, 0, 0],
        [3, 0, 0, 7],
        [0, 0, 2, 0],
    ]
    A = TripletMatrix.from_dense(dense)

    AT1 = A.transpose_slow()
    AT2 = A.transpose_fast()

    # 验证一致
    assert AT1.to_dense() == AT2.to_dense()
    # 打印结果
    print("A^T (slow):", AT1.data)
    print("A^T (fast):", AT2.data)

    A_dense = [
        [1, 0],
        [0, 2],
        [3, 0],
    ]
    # 矩阵 B: 2×3
    B_dense = [
        [0, 4, 0],
        [5, 0, 6],
    ]

    A = TripletMatrix.from_dense(A_dense)
    B = TripletMatrix.from_dense(B_dense)

    C = A.matmul(B)

    print("A (dense) =")
    for row in A.to_dense(): print(row)
    print("\nB (dense) =")
    for row in B.to_dense(): print(row)
    print("\nC = A * B (dense) =")
    for row in C.to_dense(): print(row)

    # 输出 C 三元组形式
    print("\nC (triplets) =", C.data)

A^T (slow): [Triple(r=0, c=1, v=3), Triple(r=1, c=0, v=5), Triple(r=2, c=2, v=2), Triple(r=3, c=1, v=7)]
A^T (fast): [Triple(r=0, c=1, v=3), Triple(r=1, c=0, v=5), Triple(r=2, c=2, v=2), Triple(r=3, c=1, v=7)]
A (dense) =
[1, 0]
[0, 2]
[3, 0]

B (dense) =
[0, 4, 0]
[5, 0, 6]

C = A * B (dense) =
[0, 4, 0]
[10, 0, 12]
[0, 12, 0]

C (triplets) = [Triple(r=0, c=1, v=4), Triple(r=1, c=0, v=10), Triple(r=1, c=2, v=12), Triple(r=2, c=1, v=12)]
